In [63]:
from __future__ import annotations
from contextlib import contextmanager
from typing import Any, Iterable, Sequence
import pandas as pd
from mysql.connector.pooling import MySQLConnectionPool
import os
from dotenv import load_dotenv
from tqdm import tqdm
import ast
from datetime import datetime, date, time, timedelta
import numpy as np

In [64]:
class DatabaseManager:
    """
    Optimizes the initializaiton of a MySQLConnectionPool with UTF-8MB4 encoding.

    Usage Example:
    db = DatabaseManager(
        host="localhost",
        user="admin",
        password="secret",
        database="production"
    )

    Select example
    df = db.select("SELECT * FROM users WHERE status = %s", ("active",))

    Insert example
    affected = db.execute("INSERT INTO logs (event) VALUES (%s)", ("startup",))

    Batch insert
    affected = db.execute(
        "INSERT INTO metrics (key, value) VALUES (%s, %s)",
        [("cpu", 0.93), ("ram", 0.72)],
        many=True
    )
    """
    def __init__(
        self,
        host: str,
        user: str,
        password: str,
        database: str,
        port: int = 3306,
        pool_name: str = "db_pool",
        pool_size: int = 6,
    ) -> None:
        self._pool: MySQLConnectionPool = MySQLConnectionPool(
            pool_name=pool_name,
            pool_size=pool_size,
            host=host,
            port=port,
            user=user,
            password=password,
            database=database,
            charset="utf8mb4",
            autocommit=False,
        )

    @contextmanager
    def _connection(self):
        conn = self._pool.get_connection()
        try:
            yield conn
            conn.commit()
        except Exception:
            conn.rollback()
            raise
        finally:
            conn.close()

    @contextmanager
    def _cursor(self, conn):
        cur = conn.cursor()
        try:
            yield cur
        finally:
            cur.close()

    def select(self, sql: str, params: Sequence[Any] | None = None) -> pd.DataFrame:
        with self._connection() as conn, self._cursor(conn) as cur:
            cur.execute(sql, params or ())
            columns = [c[0] for c in cur.description]
            return pd.DataFrame(cur.fetchall(), columns=columns)

    def execute(
        self,
        sql: str,
        params: Sequence[Any] | None = None,
        many: bool = False,
    ) -> int:
        with self._connection() as conn, self._cursor(conn) as cur:
            if many and isinstance(params, Iterable):
                cur.executemany(sql, params)  # type: ignore[arg-type]
            else:
                cur.execute(sql, params or ())
            return cur.rowcount


In [65]:
load_dotenv()

host = os.getenv('DB_HOST')
port = int(os.getenv('DB_PORT'))
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
database = os.getenv('DB_NAME')

try:
    DB = DatabaseManager(host="localhost", user=user, password="venomio", database="vpfm")
except Exception as e:
    print(f"[INFO] No se pudo conectar a la DB local: {e}")
    DB = None

In [66]:
leagues_df = DB.select("SELECT id FROM leagues WHERE is_active = 1")

for lid in tqdm(leagues_df['id'].tolist(), desc="League RAxG Coeff"):
    print(lid)

matches_query = """
SELECT 
    mg.id, 
    mg.date,
    md.teamA_players, 
    md.teamB_players, 
    md.teamA_xg,
    md.teamB_xg,
    md.minutes_played
FROM match_general mg
LEFT JOIN match_detailed md
    ON mg.id = md.match_id
WHERE league_id = %s
"""
matches_df = DB.select(matches_query, (1,))

matches_df

League RAxG Coeff: 100%|██████████| 4/4 [00:00<?, ?it/s]

1
2
3
4


,id,date,teamA_players,teamB_players,teamA_xg,teamB_xg,minutes_played
0,4458,2024-07-24 19:00:00,"[""Gustavo_1_C"", ""Rodrigo Fagundes_3_C"", ""Wilke...","[""João Ricardo Riedi_1_F"", ""Guilherme Tinga_2_...",0.03,0.00,6
1,4458,2024-07-24 19:00:00,"[""Gustavo_1_C"", ""Rodrigo Fagundes_3_C"", ""Wilke...","[""João Ricardo Riedi_1_F"", ""Guilherme Tinga_2_...",0.00,0.06,6
2,4458,2024-07-24 19:00:00,"[""Gustavo_1_C"", ""Rodrigo Fagundes_3_C"", ""Wilke...","[""João Ricardo Riedi_1_F"", ""Guilherme Tinga_2_...",0.03,0.02,4
3,4458,2024-07-24 19:00:00,"[""Gustavo_1_C"", ""Rodrigo Fagundes_3_C"", ""Wilke...","[""João Ricardo Riedi_1_F"", ""Guilherme Tinga_2_...",0.00,0.04,5
4,4458,2024-07-24 19:00:00,"[""Gustavo_1_C"", ""Rodrigo Fagundes_3_C"", ""Wilke...","[""João Ricardo Riedi_1_F"", ""Guilherme Tinga_2_...",0.03,0.32,7
...,...,...,...,...,...,...,...
4436,6377,2025-06-01 19:30:00,"[""Cássio_1_C"", ""Kaiki_6_C"", ""Matheus Pereira_1...","[""Wéverton_21_P"", ""Bruno Fuchs_3_P"", ""Agustín ...",0.01,0.06,15
4437,6377,2025-06-01 19:30:00,"[""Cássio_1_C"", ""Kaiki_6_C"", ""Matheus Pereira_1...","[""Wéverton_21_P"", ""Bruno Fuchs_3_P"", ""Agustín ...",0.06,0.04,15
4438,6377,2025-06-01 19:30:00,"[""Cássio_1_C"", ""Kaiki_6_C"", ""Matheus Pereira_1...","[""Wéverton_21_P"", ""Bruno Fuchs_3_P"", ""Agustín ...",0.58,0.00,11
4439,6377,2025-06-01 19:30:00,"[""Cássio_1_C"", ""Kaiki_6_C"", ""Matheus Pereira_1...","[""Wéverton_21_P"", ""Bruno Fuchs_3_P"", ""Agustín ...",0.41,0.00,2


In [67]:
matches_df['date'] = pd.to_datetime(matches_df['date'])
matches_df['days_ago'] = (datetime.now() - matches_df['date']).dt.days

matches_df['teamA_players'] = matches_df['teamA_players'].apply(lambda v: v if isinstance(v, list) else ast.literal_eval(v))
matches_df['teamB_players'] = matches_df['teamB_players'].apply(lambda v: v if isinstance(v, list) else ast.literal_eval(v))
matches_df['time_weight'] = np.exp(-np.log(2) * matches_df['days_ago'] / 180)
total_weight = matches_df['time_weight'].sum()
matches_df['time_weight'] = matches_df['time_weight'] / total_weight * len(matches_df)

players_set = set()
for idx, row in matches_df.iterrows():
    teamA = row['teamA_players'] if isinstance(row['teamA_players'], list) else [row['teamA_players']]
    teamB = row['teamB_players'] if isinstance(row['teamB_players'], list) else [row['teamB_players']]
    players_set.update(teamA)
    players_set.update(teamB)

players = sorted(list(players_set))
print(len(players))

948


In [70]:
for idx, row in matches_df.iterrows():
    minutes = row['minutes_played']
    if minutes == 0:
        continue

    teamA_players = row['teamA_players'] if isinstance(row['teamA_players'], list) else [row['teamA_players']]
    teamB_players = row['teamB_players'] if isinstance(row['teamB_players'], list) else [row['teamB_players']]

    teamA_xg = row['teamA_xg']
    teamB_xg = row['teamB_xg']
    print(teamA_players)

['Gustavo_1_C', 'Rodrigo Fagundes_3_C', 'Wilker Ángel_4_C', 'Higor Meritão_5_C', 'Fellipe Mateus_7_C', 'Newton_8_C', 'Jonathan_13_C', 'Miguel Trauco_14_C', 'Marcelo Hermes_22_C', 'Yannick Bolasie_11_C', 'Éder_23_C']
['Gustavo_1_C', 'Rodrigo Fagundes_3_C', 'Wilker Ángel_4_C', 'Higor Meritão_5_C', 'Fellipe Mateus_7_C', 'Newton_8_C', 'Jonathan_13_C', 'Miguel Trauco_14_C', 'Marcelo Hermes_22_C', 'Yannick Bolasie_11_C', 'Éder_23_C']
['Gustavo_1_C', 'Rodrigo Fagundes_3_C', 'Wilker Ángel_4_C', 'Higor Meritão_5_C', 'Fellipe Mateus_7_C', 'Newton_8_C', 'Jonathan_13_C', 'Miguel Trauco_14_C', 'Marcelo Hermes_22_C', 'Yannick Bolasie_11_C', 'Éder_23_C']
['Gustavo_1_C', 'Rodrigo Fagundes_3_C', 'Wilker Ángel_4_C', 'Higor Meritão_5_C', 'Fellipe Mateus_7_C', 'Newton_8_C', 'Jonathan_13_C', 'Miguel Trauco_14_C', 'Marcelo Hermes_22_C', 'Yannick Bolasie_11_C', 'Éder_23_C']
['Gustavo_1_C', 'Rodrigo Fagundes_3_C', 'Wilker Ángel_4_C', 'Higor Meritão_5_C', 'Fellipe Mateus_7_C', 'Newton_8_C', 'Jonathan_13_C', 'M